# LSTM Regression with MPG Data
Neste notebook, vamos realizar uma regressão da coluna `mpg` usando uma rede LSTM. Vamos começar baixando os dados.

In [ ]:
!pip install -q pandas
import pandas as pd

# Baixando os dados do Gist
url = 'https://gist.githubusercontent.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv'
df = pd.read_csv(url)
df.head()

## One-Hot Encoding
Vamos aplicar one-hot encoding nas colunas categóricas para preparar os dados para a rede LSTM.

In [ ]:
# Colunas categóricas para one-hot encoding
categorical_cols = ['cyl', 'vs', 'am', 'gear', 'carb']

# Aplicando one-hot encoding
df_encoded = pd.get_dummies(df, columns=categorical_cols)
df_encoded.head()

## LSTM Regression
Agora que os dados estão preparados, vamos criar e treinar uma rede LSTM para prever a coluna `mpg`.

In [ ]:
!pip install -q tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Separando features e target
X = df_encoded.drop(['mpg', 'model'], axis=1)
y = df_encoded['mpg']

# Dividindo em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizando os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Redimensionando para o formato do LSTM
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Criando o modelo LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train_scaled.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Treinando o modelo
history = model.fit(X_train_scaled, y_train, epochs=100, verbose=0)

# Avaliando o modelo
loss = model.evaluate(X_test_scaled, y_test, verbose=0)
loss

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flytekit 1.2.11 requires grpcio!=1.45.0,<1.49.1,<2.0,>=1.43.0, but you have grpcio 1.59.0 which is incompatible.
flytekit 1.2.11 requires protobuf<4,>=3.6.1, but you have protobuf 4.21.7 which is incompatible.
dagster 1.0.15 requires grpcio<1.48.1,>=1.32.0, but you have grpcio 1.59.0 which is incompatible.
dagster 1.0.15 requires protobuf<4,>=3.13.0, but you have protobuf 4.21.7 which is incompatible.


2023-10-16 00:08:37.825244: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-16 00:08:37.868965: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-16 00:08:37.869015: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-16 00:08:37.869039: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-16 00:08:37.876342: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-16 00:08:37.876749: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

2023-10-16 00:08:38.763843: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


106.1802978515625

## Random Forest como Modelo de Comparação
Vamos usar uma Random Forest para testar a performance contra uma LSTM.

In [ ]:
!pip install -q scikit-learn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Criando o modelo Random Forest
rf_model = RandomForestRegressor(random_state=42)

# Treinando o modelo com os dados não normalizados
rf_model.fit(X_train, y_train)

# Fazendo previsões
y_pred = rf_model.predict(X_test)

# Calculando o erro quadrático médio
rf_mse = mean_squared_error(y_test, y_pred)
rf_mse

4.82531699999996

## Usando XGBoost
Vamos treinar e avaliar usando XGBoost para comparar suas performances com outros modelos


In [ ]:
from xgboost import XGBRegressor

# Criando e treinando o modelo XGBoost
xgb_model = XGBRegressor(random_state=42)
xgb_model.fit(X_train, y_train)

# Fazendo previsões com XGBoost
y_pred_xgb = xgb_model.predict(X_test)

# Calculando o erro quadrático médio para XGBoost
xgb_mse = mean_squared_error(y_test, y_pred_xgb)
xgb_mse

7.1797414008511184

## MLP (Multilayer Perceptron) com One-Hot Encoding
Vamos criar e treinar uma MLP simples para comparar sua performance com os outros modelos.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Normalizando os dados para a MLP
X_train_scaled_mlp = scaler.transform(X_train)
X_test_scaled_mlp = scaler.transform(X_test)

# Criando o modelo MLP
mlp_model = Sequential()
mlp_model.add(Dense(64, activation='relu', input_shape=(X_train_scaled_mlp.shape[1],)))
mlp_model.add(Dense(32, activation='relu'))
mlp_model.add(Dense(1))

# Compilando o modelo
mlp_model.compile(optimizer='adam', loss='mse')

# Treinando o modelo
mlp_history = mlp_model.fit(X_train_scaled_mlp, y_train, epochs=100, verbose=0)

# Avaliando o modelo
mlp_loss = mlp_model.evaluate(X_test_scaled_mlp, y_test, verbose=0)
mlp_loss

4.7747039794921875

## CatBoost (Categorical Boosting)
Vamos testar o CatBoost, um algoritmo de boosting que lida bem com variáveis categóricas, para ver como ele se compara aos outros modelos.

In [ ]:
!pip install -q catboost
from catboost import CatBoostRegressor

# Criando e treinando o modelo CatBoost
cat_model = CatBoostRegressor(verbose=0, random_state=42)
cat_model.fit(X_train, y_train)

# Fazendo previsões com CatBoost
y_pred_cat = cat_model.predict(X_test)

# Calculando o erro quadrático médio para CatBoost
cat_mse = mean_squared_error(y_test, y_pred_cat)
cat_mse

5.898422515020589

## Modelos Lineares Regularizados
Vamos testar alguns modelos lineares regularizados, como Lasso e Ridge, para ver como eles se comparam aos modelos anteriores.

In [ ]:
from sklearn.linear_model import Lasso, Ridge

# Criando e treinando o modelo Lasso
lasso_model = Lasso(alpha=1.0, random_state=42)
lasso_model.fit(X_train, y_train)

# Fazendo previsões com Lasso
y_pred_lasso = lasso_model.predict(X_test)

# Calculando o erro quadrático médio para Lasso
lasso_mse = mean_squared_error(y_test, y_pred_lasso)
lasso_mse

12.221403174268389

In [ ]:
# Criando e treinando o modelo Ridge
ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_model.fit(X_train, y_train)

# Fazendo previsões com Ridge
y_pred_ridge = ridge_model.predict(X_test)

# Calculando o erro quadrático médio para Ridge
ridge_mse = mean_squared_error(y_test, y_pred_ridge)
ridge_mse

6.544929303826807

## Otimização de Hiperparâmetros para Random Forest
Vamos usar a busca em grade (Grid Search) para otimizar os hiperparâmetros da Random Forest e ver se conseguimos melhorar ainda mais o desempenho.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definindo os parâmetros para a busca em grade
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Criando o modelo para Grid Search
rf_grid = GridSearchCV(estimator=RandomForestRegressor(random_state=42), param_grid=param_grid, cv=3, verbose=2, n_jobs=-1)

# Realizando a busca em grade
rf_grid.fit(X_train, y_train)

# Melhores parâmetros encontrados
best_params = rf_grid.best_params_
best_params

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s


[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   0.1s


[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.0s


[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s


[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=150; total time=   0.1s


...

[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=5, n_estimators=150; total time=   0.1s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=5, n_estimators=150; total time=   0.1s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=10, n_estimators=50; total time=   0.0s


[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.1s


[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=10, n_estimators=150; total time=   0.1s


[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=10, n_estimators=150; total time=   0.1s
[CV] END max_depth=30, min_samples_leaf=4, min_samples_split=10, n_estimators=150; total time=   0.1s


{'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 100}

In [ ]:
# Criando e treinando o modelo Random Forest otimizado
best_rf_model = RandomForestRegressor(n_estimators=best_params['n_estimators'],
                                      max_depth=best_params['max_depth'],
                                      min_samples_split=best_params['min_samples_split'],
                                      min_samples_leaf=best_params['min_samples_leaf'],
                                      random_state=42)
best_rf_model.fit(X_train, y_train)

# Fazendo previsões com o modelo otimizado
y_pred_best_rf = best_rf_model.predict(X_test)

# Calculando o erro quadrático médio para o modelo otimizado
best_rf_mse = mean_squared_error(y_test, y_pred_best_rf)
best_rf_mse

5.567781366228249